In [3]:
import os

In [4]:
#!pip install fastchat

In [5]:
import csv
import json
from fastchat.conversation import Conversation, SeparatorStyle
import pandas as pd

In [6]:
this_nb_dir = os.getcwd() 

In [7]:
from glob import glob

In [8]:
import json

def read_jsonl(file_path):
    """Read a JSONL file and return a list of valid JSON objects."""
    lines = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                lines.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Skipping invalid line in {file_path}: {e}")
    return lines


In [62]:
combined_lines = []
for file in glob(this_nb_dir+"/../datasets/qa_pairs/*.jsonl"):
    combined_lines.extend(read_jsonl(file))

In [10]:
output_json_filepath = os.getcwd() + f'/../datasets/finetune_data.jsonl'

In [ ]:
import pandas as pd
import json
from fastchat.conversation import Conversation, SeparatorStyle

In [63]:
with open(output_json_filepath, 'w') as jsonl_file:
    for line in combined_lines:
        # Accessing the instruction and response based on known keys
        #instruction = line["instruction"]
        #response = line["response"]
        instruction, response = line[list(line.keys())[0]],line[list(line.keys())[1]]
        conv = Conversation(
            name="alpaca",
            system_message="Below is an instruction that describes a task. Write a response that appropriately completes the request.",
            roles=("### Instruction", "### Response"),
            sep_style=SeparatorStyle.ADD_COLON_TWO,
            sep="\n\n",
            sep2="</s>",
            messages=[]
        )
        
        # Append messages correctly
        conv.append_message("### Instruction", instruction)
        conv.append_message("### Response", response)
        
        # Get the prompt from the conversation
        prompt = conv.get_prompt()
        
        # Write the prompt to the JSONL file
        json_line = json.dumps({"prompt": prompt})
        jsonl_file.write(json_line + '\n')

# autotrain wants a CSV that has it all concatenated as a "text" column

In [ ]:
with open(output_json_filepath, 'w') as f:
    for row in combined_lines:
        

In [55]:
output_csv_filepath = os.getcwd() + f'/../datasets/finetune_data.csv'

In [60]:
with open(output_csv_filepath, 'w') as f:
    f.write('blank,text\n')
    for row in combined_lines:
        try:
            q = row['question'].replace('"',"'")
            a = row['answer'].replace('"',"'")
            conversation = f',"you are a Gradio ui ai assistant do use your knowledge to respond to this request accurately\n\n### Instruction:\n{q} \n\n### Response:\n{a}"' 
                
            f.write(conversation + '\n')
        except:
            print(row)

check that it can be read locally

In [61]:

pd.read_csv(output_csv_filepath)

,blank,text
0,NaN,you are a Gradio ui ai assistant do use your k...
1,NaN,you are a Gradio ui ai assistant do use your k...
2,NaN,you are a Gradio ui ai assistant do use your k...
3,NaN,you are a Gradio ui ai assistant do use your k...
4,NaN,you are a Gradio ui ai assistant do use your k...
...,...,...
932,NaN,you are a Gradio ui ai assistant do use your k...
933,NaN,you are a Gradio ui ai assistant do use your k...
934,NaN,you are a Gradio ui ai assistant do use your k...
935,NaN,you are a Gradio ui ai assistant do use your k...


# Trying Predibase Mistral 7b 
wants this:
{"prompt": ..., "completion": ...}
{"prompt": "Please summarize the following article ...", "completion": "Madonna kicks off Celebration World Tour in London"}
{"prompt": "Please summarize the following article ...", "completion": "Facebook Releases First Transparency Report on what Americans see on the platform"}
{"prompt": ..., "completion": ...}

In [67]:
line = combined_lines[0]

In [72]:
str(line).replace("'question'",'"prompt"').replace("'answer'",'"response"')

'{"prompt": "\\nI have an English-to-Russian translation model and the ruDALL-E text-to-image generation model. With my current code, I can input English text, translate it to Russian, feed the Russian text to ruDALL-E to generate an image, and output the generated image.\\n\\nI would like to create a Gradio web interface to demonstrate this pipeline end-to-end:\\n- Have a text input box where the user can enter an English text prompt \\n- When the user submits, first translate the English text to Russian\\n- Display the translated Russian text\\n- Feed the Russian text to ruDALL-E to generate an image \\n- Display the generated image in the interface\\n- Allow the user to resubmit the form to generate new images from the same prompt\\n- Provide some pre-filled example prompts the user can try\\n- Put the translation and image generation components side-by-side in the interface\\n- Launch the interface with a Hugging Face theme\\n- Include links to the model\'s GitHub repo and a relate

{'prompt': "\nI have an English-to-Russian translation model and the ruDALL-E text-to-image generation model. With my current code, I can input English text, translate it to Russian, feed the Russian text to ruDALL-E to generate an image, and output the generated image.\n\nI would like to create a Gradio web interface to demonstrate this pipeline end-to-end:\n- Have a text input box where the user can enter an English text prompt \n- When the user submits, first translate the English text to Russian\n- Display the translated Russian text\n- Feed the Russian text to ruDALL-E to generate an image \n- Display the generated image in the interface\n- Allow the user to resubmit the form to generate new images from the same prompt\n- Provide some pre-filled example prompts the user can try\n- Put the translation and image generation components side-by-side in the interface\n- Launch the interface with a Hugging Face theme\n- Include links to the model's GitHub repo and a related article\n",
 

In [77]:
output_json_filepath_predibase = os.getcwd() + f'/../datasets/finetune_data_mistral_predibase.jsonl'

In [80]:
with open(output_json_filepath_predibase, 'w') as jsonl_file:
    for line in combined_lines:
        new_line = {}
        new_line["prompt"] = line['question']
        new_line["completion"] = line['answer']
        jsonl_file.write(json.dumps(new_line) + '\n')

In [73]:
line.keys()

dict_keys(['question', 'answer'])